# NaonedIA - Expérience Logement

Hector Basset - Ippon Technologies

Analyse des données du cadastre en vue de la sélection d'un dataset.

In [1]:
import pandas as pd
import numpy as np
from os import path

import plotly.offline as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

In [2]:
py.init_notebook_mode(connected=True)

def py_table(data, filename):
    table = ff.create_table(data)
    py.iplot(table, filename=filename)

def py_pie(values, title, filename, labels=None):
    if labels is None:
        labels = values.index
        values = values.values
    pie = go.Pie(labels=labels, values=values, title=title)
    py.iplot([pie], filename=filename)

In [3]:
def get_communes():
    return pd.read_csv(
        'data/code-postaux.csv',
        delimiter=';',
        header=0,
        usecols=[
            'Code_commune_INSEE'
        ],
        index_col=False,
        squeeze=True
    ).unique()

def get_data_year(year, communes):
    data_year = pd.read_csv(
        'data/dvf/44_%i.csv.xz' % year,
        header=0,
        usecols=[
            'date_mutation',
            'nature_mutation',
            'valeur_fonciere',
            'code_postal',
            'nom_commune',
            'ancien_code_commune',
            'code_commune',
            'lot1_surface_carrez',
            'lot2_surface_carrez',
            'lot3_surface_carrez',
            'lot4_surface_carrez',
            'lot5_surface_carrez',
            'nombre_lots',
            'type_local',
            'surface_reelle_bati',
            'nombre_pieces_principales',
            'surface_terrain',
            'longitude',
            'latitude'
        ],
        index_col=False
    )

    # Suppression des observations qui ne sont pas dans Nantes Métropole
    data_year.drop(data_year[~(data_year['code_commune'].isin(communes) | data_year['ancien_code_commune'].isin(communes))].index, inplace=True)

    # Suppression des variables code commune INSEE
    data_year.drop(['code_commune', 'ancien_code_commune'],axis=1, inplace=True)

    return data_year

def get_data():
    communes = get_communes()

    data = pd.concat([get_data_year(year, communes) for year in [2014, 2015, 2016, 2017, 2018]], ignore_index=True)

    data['nature_mutation'].replace('nan', np.nan, inplace=True)
    data['nature_mutation'] = data['nature_mutation'].astype('category')
    data['valeur_fonciere'] = data['valeur_fonciere'].astype('float64')
    data['code_postal'] = data['code_postal'].astype('str')
    data['code_postal'] = data['code_postal'].apply(lambda n: n.split('.')[0])
    data['code_postal'].replace('nan', np.nan, inplace=True)
    data['code_postal'] = data['code_postal'].astype('category')
    data['nom_commune'].replace('nan', np.nan, inplace=True)
    data['nom_commune'] = data['nom_commune'].astype('category')
    data['lot1_surface_carrez'] = data['lot1_surface_carrez'].astype('float64')
    data['lot2_surface_carrez'] = data['lot2_surface_carrez'].astype('float64')
    data['lot3_surface_carrez'] = data['lot3_surface_carrez'].astype('float64')
    data['lot4_surface_carrez'] = data['lot4_surface_carrez'].astype('float64')
    data['lot5_surface_carrez'] = data['lot5_surface_carrez'].astype('float64')
    data['nombre_lots'] = data['nombre_lots'].astype('int64')
    data['type_local'].replace('nan', np.nan, inplace=True)
    data['type_local'] = data['type_local'].astype('category')
    data['surface_reelle_bati'] = data['surface_reelle_bati'].astype('float64')
    data['nombre_pieces_principales'] = data['nombre_pieces_principales'].astype('float64')
    data['surface_terrain'] = data['surface_terrain'].astype('float64')
    data['longitude'] = data['longitude'].astype('float64')
    data['latitude'] = data['latitude'].astype('float64')

    return data

## Chargement des données relatives à Nantes Métropole

In [4]:
df = None
if path.isfile('data/checkpoints/all.pkl.xz'):
    df = pd.read_pickle('data/checkpoints/all.pkl.xz')
else:
    df = get_data()
    df.to_pickle('data/checkpoints/all.pkl.xz')

print("%i observations chargées" % df.shape[0])
df.head()

151591 observations chargées


,date_mutation,nature_mutation,valeur_fonciere,code_postal,nom_commune,lot1_surface_carrez,lot2_surface_carrez,lot3_surface_carrez,lot4_surface_carrez,lot5_surface_carrez,nombre_lots,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude
0,2014-01-08,Vente,29000.00,44000,Nantes,NaN,NaN,NaN,NaN,NaN,0,Dépendance,NaN,0.0,26.0,-1.545448,47.221956
1,2014-01-08,Vente,29000.00,44000,Nantes,NaN,NaN,NaN,NaN,NaN,0,Dépendance,NaN,0.0,26.0,-1.545448,47.221956
2,2014-01-09,Vente en l'état futur d'achèvement,285000.00,44000,Nantes,NaN,NaN,NaN,NaN,NaN,1,Dépendance,NaN,0.0,NaN,NaN,NaN
3,2014-01-09,Vente en l'état futur d'achèvement,285000.00,44000,Nantes,NaN,NaN,NaN,NaN,NaN,1,Appartement,65.0,3.0,NaN,NaN,NaN
4,2014-01-06,Vente en l'état futur d'achèvement,251538.46,NaN,Nantes,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,-1.533561,47.216841


In [5]:
print("%i variables" % df.shape[1])
pd.DataFrame(df.columns, columns=['Colonnes'])

17 variables


,Colonnes
0,date_mutation
1,nature_mutation
2,valeur_fonciere
3,code_postal
4,nom_commune
5,lot1_surface_carrez
6,lot2_surface_carrez
7,lot3_surface_carrez
8,lot4_surface_carrez
9,lot5_surface_carrez


## Analyse des natures de mutation

In [6]:
py_pie(df.groupby(['nature_mutation']).size(), 'Natures de mutation dans le dataset', 'natures')

Ce dataset inclue des mutations autres qu'une simple vente (qui représente cependant la grande majorité des cas). Pour simplifier et se limiter aux cas d'utilisation les plus fréquents, nous n'allons garder que les ventes, et retirer la variable `nature_mutation`.

In [7]:
df.drop(df[df['nature_mutation'] != 'Vente'].index, inplace=True)
df.drop('nature_mutation', axis=1, inplace=True)

In [8]:
print("%i observations restantes" % df.shape[0])
df.head()

113336 observations restantes


,date_mutation,valeur_fonciere,code_postal,nom_commune,lot1_surface_carrez,lot2_surface_carrez,lot3_surface_carrez,lot4_surface_carrez,lot5_surface_carrez,nombre_lots,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude
0,2014-01-08,29000.0,44000,Nantes,NaN,NaN,NaN,NaN,NaN,0,Dépendance,NaN,0.0,26.0,-1.545448,47.221956
1,2014-01-08,29000.0,44000,Nantes,NaN,NaN,NaN,NaN,NaN,0,Dépendance,NaN,0.0,26.0,-1.545448,47.221956
8,2014-01-08,192.0,44800,Saint-Herblain,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,24.0,-1.629867,47.239388
9,2014-01-02,194400.0,44100,Nantes,NaN,NaN,NaN,NaN,NaN,1,Dépendance,NaN,0.0,NaN,-1.603261,47.209692
10,2014-01-02,194400.0,44100,Nantes,NaN,NaN,NaN,NaN,NaN,1,Appartement,84.0,4.0,NaN,-1.603261,47.209692


## Analyse des types de locaux

In [9]:
py_pie(df.groupby(['type_local']).size(), 'Types de locaux dans le dataset', 'locaux')

Là aussi, pour simplifier et adresser le plus grand nombre de cas d'utilisation, nous allons nous limiter aux appartements et aux maisons.

In [10]:
df.drop(df[~df['type_local'].isin(['Appartement', 'Maison'])].index, inplace=True)
df['type_local'].cat.remove_unused_categories(inplace=True)

In [11]:
print("%i observations restantes" % df.shape[0])
df.head()

57567 observations restantes


,date_mutation,valeur_fonciere,code_postal,nom_commune,lot1_surface_carrez,lot2_surface_carrez,lot3_surface_carrez,lot4_surface_carrez,lot5_surface_carrez,nombre_lots,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude
10,2014-01-02,194400.0,44100,Nantes,NaN,NaN,NaN,NaN,NaN,1,Appartement,84.0,4.0,NaN,-1.603261,47.209692
12,2014-01-02,107000.0,44800,Saint-Herblain,45.80,NaN,NaN,NaN,NaN,1,Appartement,46.0,2.0,NaN,-1.644254,47.207462
13,2014-01-08,295000.0,44000,Nantes,NaN,89.2,NaN,NaN,NaN,2,Appartement,105.0,2.0,NaN,-1.558665,47.216404
14,2014-01-09,208154.0,44300,Nantes,103.64,NaN,NaN,NaN,NaN,1,Appartement,103.0,5.0,NaN,-1.519753,47.272364
17,2014-01-06,79000.0,44000,Nantes,26.45,NaN,NaN,NaN,NaN,1,Appartement,25.0,1.0,NaN,-1.551322,47.216626


## Analyse du nombre de lots

In [12]:
py_pie(df.groupby(['nombre_lots']).size(), 'Nombres de lots de chaque bien dans le dataset', 'lots')

De nombreux biens sont vendus en lots, et la valeur foncière représente alors le total des lots. Là aussi dans un but de simplification nous n'allons garder que les biens contenant 0 ou 1 lot, et retirer les variables `nombre_lots` et `lot2_surface_carrez` et plus. Nous renommons du coup `lot1_surface_carrez` en `surface_carrez`.

In [13]:
df.drop(df[df['nombre_lots'] > 1].index, inplace=True)
df.drop('nombre_lots', axis=1, inplace=True)
df.drop('lot2_surface_carrez', axis=1, inplace=True)
df.drop('lot3_surface_carrez', axis=1, inplace=True)
df.drop('lot4_surface_carrez', axis=1, inplace=True)
df.drop('lot5_surface_carrez', axis=1, inplace=True)
df.rename({'lot1_surface_carrez': 'surface_carrez'}, axis=1, inplace=True)

In [14]:
print("%i observations restantes" % df.shape[0])
df.head()

42196 observations restantes


,date_mutation,valeur_fonciere,code_postal,nom_commune,surface_carrez,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude
10,2014-01-02,194400.0,44100,Nantes,NaN,Appartement,84.0,4.0,NaN,-1.603261,47.209692
12,2014-01-02,107000.0,44800,Saint-Herblain,45.80,Appartement,46.0,2.0,NaN,-1.644254,47.207462
14,2014-01-09,208154.0,44300,Nantes,103.64,Appartement,103.0,5.0,NaN,-1.519753,47.272364
17,2014-01-06,79000.0,44000,Nantes,26.45,Appartement,25.0,1.0,NaN,-1.551322,47.216626
21,2014-01-02,335000.0,44300,Nantes,NaN,Maison,118.0,5.0,562.0,-1.506824,47.231122


## Suppression des doublons

In [15]:
df.drop_duplicates(inplace=True)

In [16]:
print("%i observations restantes" % df.shape[0])
df.head()

40275 observations restantes


,date_mutation,valeur_fonciere,code_postal,nom_commune,surface_carrez,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude
10,2014-01-02,194400.0,44100,Nantes,NaN,Appartement,84.0,4.0,NaN,-1.603261,47.209692
12,2014-01-02,107000.0,44800,Saint-Herblain,45.80,Appartement,46.0,2.0,NaN,-1.644254,47.207462
14,2014-01-09,208154.0,44300,Nantes,103.64,Appartement,103.0,5.0,NaN,-1.519753,47.272364
17,2014-01-06,79000.0,44000,Nantes,26.45,Appartement,25.0,1.0,NaN,-1.551322,47.216626
21,2014-01-02,335000.0,44300,Nantes,NaN,Maison,118.0,5.0,562.0,-1.506824,47.231122


## Données manquantes

Observons le taux de remplissage des colonnes :

In [17]:
pd.DataFrame((df.count() / df.shape[0] * 100).apply(lambda p: "%.2f %%" % p), columns=['Taux de remplissage'])

,Taux de remplissage
date_mutation,100.00 %
valeur_fonciere,99.69 %
code_postal,100.00 %
nom_commune,100.00 %
surface_carrez,29.71 %
type_local,100.00 %
surface_reelle_bati,100.00 %
nombre_pieces_principales,100.00 %
surface_terrain,58.87 %
longitude,96.77 %


La valeur foncière n'est pas toujours remplie, or il s'agit de la variable que l'on cherche à prédire. On supprime donc les observations concernées.

In [18]:
df.drop(df[df['valeur_fonciere'].isna()].index, inplace=True)

In [19]:
print("%i observations restantes" % df.shape[0])
df.head()

40150 observations restantes


,date_mutation,valeur_fonciere,code_postal,nom_commune,surface_carrez,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude
10,2014-01-02,194400.0,44100,Nantes,NaN,Appartement,84.0,4.0,NaN,-1.603261,47.209692
12,2014-01-02,107000.0,44800,Saint-Herblain,45.80,Appartement,46.0,2.0,NaN,-1.644254,47.207462
14,2014-01-09,208154.0,44300,Nantes,103.64,Appartement,103.0,5.0,NaN,-1.519753,47.272364
17,2014-01-06,79000.0,44000,Nantes,26.45,Appartement,25.0,1.0,NaN,-1.551322,47.216626
21,2014-01-02,335000.0,44300,Nantes,NaN,Maison,118.0,5.0,562.0,-1.506824,47.231122


Certains biens n'ont pas pu être géolocalisés, on les supprime donc également.

In [20]:
df.drop(df[df['longitude'].isna() | df['latitude'].isna()].index, inplace=True)

In [21]:
print("%i observations restantes" % df.shape[0])
df.head()

38865 observations restantes


,date_mutation,valeur_fonciere,code_postal,nom_commune,surface_carrez,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude
10,2014-01-02,194400.0,44100,Nantes,NaN,Appartement,84.0,4.0,NaN,-1.603261,47.209692
12,2014-01-02,107000.0,44800,Saint-Herblain,45.80,Appartement,46.0,2.0,NaN,-1.644254,47.207462
14,2014-01-09,208154.0,44300,Nantes,103.64,Appartement,103.0,5.0,NaN,-1.519753,47.272364
17,2014-01-06,79000.0,44000,Nantes,26.45,Appartement,25.0,1.0,NaN,-1.551322,47.216626
21,2014-01-02,335000.0,44300,Nantes,NaN,Maison,118.0,5.0,562.0,-1.506824,47.231122


In [22]:
pd.DataFrame((df.count() / df.shape[0] * 100).apply(lambda p: "%.2f %%" % p), columns=['Taux de remplissage'])

,Taux de remplissage
date_mutation,100.00 %
valeur_fonciere,100.00 %
code_postal,99.99 %
nom_commune,100.00 %
surface_carrez,30.31 %
type_local,100.00 %
surface_reelle_bati,100.00 %
nombre_pieces_principales,100.00 %
surface_terrain,58.38 %
longitude,100.00 %


Certains bien n'ont pas de code postal, on les supprime également.

In [23]:
df.drop(df[df['code_postal'].isna()].index, inplace=True)

In [24]:
print("%i observations restantes" % df.shape[0])
df.head()

38863 observations restantes


,date_mutation,valeur_fonciere,code_postal,nom_commune,surface_carrez,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude
10,2014-01-02,194400.0,44100,Nantes,NaN,Appartement,84.0,4.0,NaN,-1.603261,47.209692
12,2014-01-02,107000.0,44800,Saint-Herblain,45.80,Appartement,46.0,2.0,NaN,-1.644254,47.207462
14,2014-01-09,208154.0,44300,Nantes,103.64,Appartement,103.0,5.0,NaN,-1.519753,47.272364
17,2014-01-06,79000.0,44000,Nantes,26.45,Appartement,25.0,1.0,NaN,-1.551322,47.216626
21,2014-01-02,335000.0,44300,Nantes,NaN,Maison,118.0,5.0,562.0,-1.506824,47.231122


In [25]:
pd.DataFrame((df.count() / df.shape[0] * 100).apply(lambda p: "%.2f %%" % p), columns=['Taux de remplissage'])

,Taux de remplissage
date_mutation,100.00 %
valeur_fonciere,100.00 %
code_postal,100.00 %
nom_commune,100.00 %
surface_carrez,30.31 %
type_local,100.00 %
surface_reelle_bati,100.00 %
nombre_pieces_principales,100.00 %
surface_terrain,58.38 %
longitude,100.00 %


La surface Carrez n'est renseignée que dans moins de 30 % des cas, et la surface du terrain dans moins de 60 %.

Proposition :
* supprimer la surface Carrez, et n'utiliser que la surface batie ;
* utiliser la surface batie à la place de celle du terrain là où cette dernière n'est pas remplie.

Toutes les autres variables sont remplies à 100 %, hormis `adresse_suffixe` mais ça n'est pas grave car une adresse ne contient pas forcement un suffix.

In [26]:
# Nombre pieces principales est maintenant rempli à 100 %, on peut le convertir en int
df['nombre_pieces_principales'] = df['nombre_pieces_principales'].astype('int64')

df.head()

,date_mutation,valeur_fonciere,code_postal,nom_commune,surface_carrez,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude
10,2014-01-02,194400.0,44100,Nantes,NaN,Appartement,84.0,4,NaN,-1.603261,47.209692
12,2014-01-02,107000.0,44800,Saint-Herblain,45.80,Appartement,46.0,2,NaN,-1.644254,47.207462
14,2014-01-09,208154.0,44300,Nantes,103.64,Appartement,103.0,5,NaN,-1.519753,47.272364
17,2014-01-06,79000.0,44000,Nantes,26.45,Appartement,25.0,1,NaN,-1.551322,47.216626
21,2014-01-02,335000.0,44300,Nantes,NaN,Maison,118.0,5,562.0,-1.506824,47.231122


In [27]:
df.to_pickle('data/checkpoints/after_analyze.pkl.xz')

## Analyse des communes et codes postaux

In [28]:
py_pie(df.groupby(['nom_commune']).size(), 'Nombres de bien par commune dans le dataset', 'communes')

In [29]:
py_pie(df.groupby(['code_postal']).size(), 'Nombres de bien par code postal dans le dataset', 'codes postaux')

## Conclusion de l'analyse

En utilisant ce dataset, un fois nettoyé, nous disposons d'une base solide de plus de 40 000 observations contenant des données réelles (non estimées par le vendeur par exemple). De plus, le taux de remplissage des variables est élevé.

Le seul bémol est le nombre un peu trop petit de variables. Cependant, il est tout à fait possible d'utiliser les informations de ce dataset pour l'enrichir et ajouter des variables qui pourraient être pertinentes. Je fais 2 propositions dans ce sens ci-dessous.

## Exploitation des adresses

Les adresses sont renseignées avec précision. Nous pourrions donc les utiliser afin d'enrichir le dataset de variables telles que : Distance de l'arrêt de tram le plus proche, de l'arrêt de bus, de la gare, d'une pharmacie, d'un supermarché, d'un groupe de commerces, etc. Le choix des variables est à arrêter en fonction des informations que l'on souhaite demander à l'utilisateur et de ce qu'il sera possible de faire pour enrichir le dataset via OpenStreetMap par exemple. Il peut également être intéressant de renseigner les distances à pied ou en voiture.

Une fois le dataset enrichi, nous supprimerons les adresses des données car le but est de ne pas stocker des informations trop précises (pour ne pas inquiéter l'utilisateur).

Pour éviter à l'utilisateur d'avoir à saisir toutes ces informations, on pourrais lui demander de saisir directement l'adresse du bien, en lui précisant bien qu'elle ne sera ni stockée ni utilisée pour la prédiction mais uniquement pour déduire les informations citées plus haut (et avec plus de précision que si c'est lui qui le fait du coup).

Nantes Métropole fournit de nombreux datasets en open data que l'on pourrai utiliser :
* [localisation des arrêts de tram et de bus](https://data.nantesmetropole.fr/explore/dataset/244400404_tan-arrets/table/) ;
* [localisation des entreprises](https://data.nantesmetropole.fr/explore/dataset/244400404_base-sirene-entreprises-nantes-metropole/table/) ;
* ...

## Exploitation de la date

Les dates de vente sont elles aussi renseignées avec précision. Nous pourrions les utiliser afin d'enrichir le dataset de variables telles que : prix moyen du mètre carré à cette date, tension du marché à cette date, revenu moyen par habitant à cette date, densité de population, etc. À renseigner pour l'année courante ou le mois courant (ou précédent), au niveau national ou local en fonction de ce que l'on peut trouver comme historique sur le net.

L'utilisateur n'aura bien entendu pas à rentrer ces données lorsqu'il voudra estimer un bien, mais on viendrait automatiquement enrichir sa saisie avec les derniers chiffres connus.